# Numerical Integration and Error Computations

Suppose that we have a set of mesh nodes $a = x_0 < x_1 < \cdots < x_N = b$ that lie in a closed and bounded interval $[a,b]$.
Also suppose that we have a continuous function $f:[a,b] \to \mathbb R$ as well as a vector $\vec v \in \mathbb R^N$ such that $v_i$ is an approximation to $f(x_i)$ for each $i$.
We use the vector $\vec v$ to define the continuous piecewise linear function $p:[a,b]\to\mathbb R$ with respect to the mesh nodes.
That is, for each subinterval $[x_i,x_{i+1}]$, $p|_{[x_i,x_{i+1}]}$ is the linear function on $[x_i,x_{i+1}]$ that connects the point $(x_i,v_i)$ to $(x_{i+1},v_{i+1})$.
We wish to compute the $L^2$ error, $H^1$ error, and $L^{\infty}$ error between $f$ and $p$, where we recall that

$$
\|f-p\|_{L^2[a,b]} = \left(\int_a^b|f(x)-p(x)|^2\,dx\right)^{1/2}
$$

$$
\|f-p\|_{H^1[a,b]} = \left(\|f-p\|_{L^2[a,b]}^2 + \|f'-p'\|_{L^2[a,b]}^2\right)^{1/2}
$$

$$
\|f-p\|_{L^\infty[a,b]} = \max\left\{|f(x)-p(x)| : x \in [a,b]\right\}
$$

To compute these, we will borrow ideas from the local-to-global assembly routines from a standard finite element code.

## The $L^2$ Error
We observe that

$$
\|f-p\|_{L^2[a,b]}^2 = \sum_{i=0}^{N-1}\int_{x_i}^{x_{i+1}}\left|f(x)-p|_{[x_i,x_{i+1}]}(x)\right|^2\,dx
$$

and we also observe that

$$
p|_{[x_i,x_{i+1}]}(x) = v_i\varphi_{i,0}(x) + v_{i+1}\varphi_{i,1}(x), \quad x \in [x_i,x_{i+1}],\ i \in \{0,\dots,N-1\}
$$

where

$$
\varphi_{i,j}(x) = \widehat \varphi_j(T_i^{-1}(x)), \quad x \in [x_i,x_{i+1}],\ i \in \{0,\dots,N-1\},\ j \in \{0,1\}
$$

$$
\widehat \varphi_0(\widehat x) = \frac{1-\widehat x}{2}, \quad \widehat x \in [-1,1]
$$

$$
\widehat \varphi_1(\widehat x) = \frac{1+\widehat x}{2}, \quad \widehat x \in [-1,1]
$$

$$
T_i(\widehat x) = \frac{x_i+x_{i+1}}{2} + \frac{x_{i+1}-x_i}{2}\widehat x, \quad \widehat x \in [-1,1], \ i \in \{0,\dots,N-1\}
$$

Therefore, upon changing variables back to the reference cell $[-1,1]$, we have that

$$
\|f-p\|_{L^2[a,b]}^2 = \sum_{i=0}^{N-1}\int_{x_i}^{x_{i+1}}\left|f(x)-p|_{[x_i,x_{i+1}]}(x)\right|^2\,dx \\
= \sum_{i=0}^{N-1}\int_{-1}^1 \left|f(T_i(\widehat x)) - v_i\widehat\varphi_0(\widehat x) - v_{i+1}\widehat\varphi_1(\widehat x)\right|^2T_i'(\widehat x)\,d\widehat x
$$

Since we have explicit formulas for $f, T_i, T_i'$ and $\widehat \varphi_j$, we can use numerical quadrature to estimate this final integral.
We will use [Gaussian quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature).
Since the $\widehat\varphi_j$ are degree 1 polynomials and we are squaring them in the integrand, we will use a quadrature rule that integrates degree 2 polynomials exactly.
From the link above, the following points and weights give such a quadrature rule:

|Points|Weights|
|---|---|
|0|8/9|
|$\sqrt{3/5}$|5/9|
|$-\sqrt{3/5}$|5/9|

Then we can estimate the integrals above by
$$
\|f-p\|_{L^2[a,b]}^2 = \sum_{i=0}^{N-1}\int_{x_i}^{x_{i+1}}\left|f(x)-p|_{[x_i,x_{i+1}]}(x)\right|^2\,dx \\
= \sum_{i=0}^{N-1}\int_{-1}^1 \left|f(T_i(\widehat x)) - v_i\widehat\varphi_0(\widehat x) - v_{i+1}\widehat\varphi_1(\widehat x)\right|^2T_i'(\widehat x)\,d\widehat x \\
\approx \sum_q \sum_{i=0}^{N-1} \left|f(T_i(\widehat x_q)) - v_i\widehat\varphi_0(\widehat x_q) - v_{i+1}\widehat\varphi_1(\widehat x_q)\right|^2T_i'(\widehat x_q)\widehat w_q
$$

where the pairs $(\widehat x_q, \widehat w_q)$ are the quadrature points and weights as in the table above.
Let us now implement this algorithm.


In [ ]:
import numpy as np

def compute_L2_error(f, xs, vs):
    # compute the L2 error \|f-p\|_{L^2[x0,xN]} with a function f that is
    # continuous on [x0,xN], nodes
    # xs = [x0 < x1 < ... < xN], and values vs = [v0, v1, ..., vN]
    # where p is the piecewise linear function that connects each (x_i, v_i) to
    # (x_{i+1}, v_{i+1})

    L2_error_squared = 0
    quad_points = [0,np.sqrt(3/5),-np.sqrt(3/5)]
    quad_weights = [8/9,5/9,5/9]

    for q in range(len(quad_points)):
        xq = quad_points[q]
        wq = quad_weights[q]

        phi_0q = (1-xq)/2
        phi_1q = (1+xq)/2

        for i in range(len(xs)-1):
            xi = xs[i]
            xi1 = xs[i+1]

            vi = vs[i]
            vi1 = vs[i+1]

            T_iq = (xi + xi1) / 2 + (xi1 - xi) / 2 * xq
            dT_idx = (xi1 - xi) / 2

            f_iq = f(T_iq)

            L2_error_squared += (f_iq - vi * phi_0q - vi1 * phi_1q)**2 * dT_idx * wq

    return np.sqrt(L2_error_squared)

## The $H^1$ Error

The $H^1$ error is computed in very much the same way as the $L^2$ error.
We just now also need the derivative of $f$ as well as the derivatives of the $\varphi_{i,j}$.
Using the chain rule, we have that

$$
\widehat\varphi_j'(\widehat x) = (\varphi_{i,j} \circ T_i)'(\widehat x) = (\varphi_{i,j})'(T_i(\widehat x))T_i'(\widehat x)
$$

so that

$$
(\varphi_{i,j})'(T_i(\widehat x)) = \widehat \varphi_j'(\widehat x) / T_i'(\widehat x).
$$

Using this, the computation proceeds as

$$
\|f'-p'\|_{L^2[a,b]}^2 = \int_a^b|f'(x) - p'(x)|^2\,dx \\
= \sum_{i=0}^{N-1}|f'(x) - (v_i\varphi_{i,0} + v_{i+1}\varphi_{i,1})'(x)|^2\,dx \\
= \sum_{i=0}^{N-1}|f'(T_i(x)) - v_i\widehat\varphi_0'(\widehat x)/T_i'(\widehat x) - v_{i+1}\varphi_1'(\widehat x)/T_i'(\widehat x)|^2T_i'(\widehat x)\,d\widehat x \\
\approx \sum_q\sum_{i=0}^{N-1}|f'(T_i(x_q)) - v_i\widehat\varphi_0'(\widehat x_q)/T_i'(\widehat x_q) - v_{i+1}\varphi_1'(\widehat x_q)/T_i'(\widehat x_q)|^2T_i'(\widehat x_q)\widehat w_q
$$

We now implement the full $H^1$ error algorithm below.

In [ ]:
def compute_H1_error(f, dfdx, xs, vs):
    # compute the H1 error \|f-p\|_{H^1[x0,xN]} with a function f that is
    # differentiable on [x0,xN] and had derivative dfdx, nodes
    # xs = [x0 < x1 < ... < xN], and values vs = [v0, v1, ..., vN]
    # where p is the piecewise linear function that connects each (x_i, v_i) to
    # (x_{i+1}, v_{i+1})

    L2_error = compute_L2_error(f, xs, vs)

    quad_points = [0,np.sqrt(3/5),-np.sqrt(3/5)]
    quad_weights = [8/9,5/9,5/9]

    dphi_0dx = -1/2
    dphi_1dx = 1/2

    L2_error_derivative_squared = 0
    for q in range(len(quad_points)):
        xq = quad_points[q]
        wq = quad_weights[q]

        for i in range(len(xs)-1):
            xi = xs[i]
            xi1 = xs[i+1]

            vi = vs[i]
            vi1 = vs[i+1]

            T_iq = (xi + xi1) / 2 + (xi1 - xi) / 2 * xq
            dT_idx = (xi1 - xi) / 2

            dfdx_iq = dfdx(T_iq)

            L2_error_derivative_squared += (dfdx_iq - vi * dphi_0dx / dT_idx - vi1 * dphi_1dx / dT_idx)**2 * dT_idx * wq

    return np.sqrt(L2_error**2 + L2_error_derivative_squared)

## $L^\infty$ Error
We proceed in a slightly different and easier manner to compute the $L^\infty$ error:

$$
\|f-p\|_{L^\infty[a,b]} = \max_{x\in[a,b]}|f(x) - p(x)| \\
= \max_{i\in\{0,\dots,N-1\}}\max_{x\in[x_i,x_{i+1}]}|f(x) - v_i\varphi_{i,0}(x) - v_{i+1}\varphi_{i,1}(x)| \\
= \max_{i\in\{0,\dots,N-1\}}\max_{\widehat x \in [-1,1]}|f(T_i(\widehat x)) - v_i\widehat \varphi_0(\widehat x) - v_{i+1}\widehat \varphi_1(\widehat x)| \\
\approx \max_q\max_{i=0,\dots,N-1}|f(T_i(\widehat x_q)) - v_i\widehat \varphi_0(\widehat x_q) - v_{i+1}\widehat \varphi_1(\widehat x_q)|
$$

We implement this below as follows.

In [ ]:
def compute_Linf_error(f, xs, vs):
    # compute the Linf error \|f-p\|_{L^inf[x0,xN]} with a function f that is
    # continuous on [x0,xN], nodes
    # xs = [x0 < x1 < ... < xN], and values vs = [v0, v1, ..., vN]
    # where p is the piecewise linear function that connects each (x_i, v_i) to
    # (x_{i+1}, v_{i+1})

    Linf_error = 0
    quad_points = [0,np.sqrt(3/5),-np.sqrt(3/5)]
    quad_weights = [8/9,5/9,5/9]

    for q in range(len(quad_points)):
        xq = quad_points[q]

        phi_0q = (1-xq)/2
        phi_1q = (1+xq)/2

        for i in range(len(xs)-1):
            xi = xs[i]
            xi1 = xs[i+1]

            vi = vs[i]
            vi1 = vs[i+1]

            T_iq = (xi + xi1) / 2 + (xi1 - xi) / 2 * xq
            f_iq = f(T_iq)

            Linf_error = max(Linf_error, abs(f_iq - vi * phi_0q - vi1 * phi_1q))

    return Linf_error

You may wish to use different quadrature rules with these computations.
In that case, you can pass the points and weights into the functions as arguments.
The code below has this as an example.

In [ ]:
def compute_Linf_error_general(f, xs, vs, quad_points):
    # compute the Linf error \|f-p\|_{L^inf[x0,xN]} with a function f that is
    # continuous on [x0,xN], nodes
    # xs = [x0 < x1 < ... < xN], and values vs = [v0, v1, ..., vN]
    # where p is the piecewise linear function that connects each (x_i, v_i) to
    # (x_{i+1}, v_{i+1})
    #
    # the user may pass in their own list of quadrature points on [-1,1]

    Linf_error = 0

    for q in range(len(quad_points)):
        xq = quad_points[q]

        phi_0q = (1-xq)/2
        phi_1q = (1+xq)/2

        for i in range(len(xs)-1):
            xi = xs[i]
            xi1 = xs[i+1]

            vi = vs[i]
            vi1 = vs[i+1]

            T_iq = (xi + xi1) / 2 + (xi1 - xi) / 2 * xq
            f_iq = f(T_iq)

            Linf_error = max(Linf_error, abs(f_iq - vi * phi_0q - vi1 * phi_1q))

    return Linf_error